# 0. Setup

In [1]:
# Libraries
import pandas as pd
import numpy as np
import copy

# Plots
import networkx as nx
import matplotlib
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots

# Dash
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
# Please specify the location of the txt file:
path = 'Data/NEW_PIT.txt'

# 1. Log parsing

#### 1.1 Extract df from text

In [3]:
def txt_to_df(path):

    # 1.1 Read Data
    f = open(path, 'r')
    content = f.read()
    content_list = content.split('/* JOBSPLIT: ')
    f.close()
    df = pd.DataFrame(content_list, columns=['Text'])

    # 1.2. Define Task type
    df['Task Type'] = df["Text"].str.split().str[0]

    # 1.3 Assign Task ID
    df = df.reset_index(drop=True)
    task_list = [0] 
    for index, row in df.iterrows():
        if (row['Task Type'] == 'TASKSTARTTIME') | (row['Task Type'] == 'JOBENDTIME'):
            task_list.append(task_list[index] +1)
        else:
            task_list.append(task_list[index])   
    df['Task ID'] = task_list[1:]

    # 1.4 Remove unessesary text
    df['Text'] = df[['Text','Task Type']].apply(lambda x: x[0].replace('\n\n','\n').replace('STEP SOURCE FOLLOWS */\n','') if x[1]=='STEP' 
                                                          else x[0].replace('*/\n','')
                                                , axis = 1)

    df['Task ID'] = df['Task ID'].astype('int')
    
    return df

In [4]:
df_log = txt_to_df(path)
df_log

,Text,Task Type,Task ID
0,,NaN,0
1,JOBSTARTTIME 27JUL2021:10:34:25.72,JOBSTARTTIME,0
2,TASKSTARTTIME 27JUL2021:10:34:25.72,TASKSTARTTIME,1
3,CATALOG INPUT WORK.SASMAC1.MTF_IFRS9_PD_PIT_BA...,CATALOG,1
4,LIBNAME WORK V9 '/opt/sas/saswork/SAS_work2019...,LIBNAME,1
...,...,...,...
3976,ELAPSED 8,ELAPSED,163
3977,PROCNAME DATASETS,PROCNAME,163
3978,proc datasets lib = work nolist noprint memty...,STEP,163
3979,JOBENDTIME 27JUL2021:10:36:37.94,JOBENDTIME,164


#### 1.2. Extract info about Code

In [5]:
def df_to_code(df):

    #-------------------------------------------
    # 2.1 Extract the Code Syntax
    df_code = df.loc[(df_log['Task Type'].isin(['STEP']))].copy()
    df_code.drop(['Task Type'], inplace=True, axis=1)
    df_code = df_code.rename(columns={'Text': 'Code'})
    df_code = df_code[['Task ID', 'Code']]

    #-------------------------------------------
    # 2.2. Prepare info about Start Time
    df_taskstarttime = df_log.loc[(df_log['Task Type'].isin(['TASKSTARTTIME']))].copy()
    df_taskstarttime['Start Time'] = df_taskstarttime['Text'].str.split().str[1]
    df_taskstarttime['Start Time'] = pd.to_datetime(df_taskstarttime['Start Time'], format='%d%b%Y:%H:%M:%S.%f')
    df_taskstarttime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.3. Prepare info about Elapsed Time
    df_elapsedtime = df_log.loc[(df_log['Task Type'].isin(['ELAPSED']))].copy()
    df_elapsedtime['Elapsed Time'] = df_elapsedtime['Text'].str.split().str[1].astype('int')/1000
    df_elapsedtime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.4. Prepare info about Procedure Names
    df_procedure = df_log.loc[(df_log['Task Type'].isin(['PROCNAME']))].copy()
    df_procedure['Procedure'] = df_procedure['Text'].str.split().str[1]
    df_procedure.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.5 Output
    df_code = df_code.merge(df_procedure, on=['Task ID'], how = 'outer')\
                     .merge(df_taskstarttime, on=['Task ID'], how = 'outer')\
                     .merge(df_elapsedtime, on=['Task ID'], how = 'outer')
    
    df_code['Start Time'] = df_code['Start Time'].astype('str')
    df_code['Task ID'] = df_code['Task ID'].astype('int')
    
    return df_code

In [6]:
df_code = df_to_code(df_log)
df_code

,Task ID,Code,Procedure,Start Time,Elapsed Time
0,1,/*--------------------------------------------...,DATASTEP,2021-07-27 10:34:25.720,0.006
1,2,data _null_;\n set __mtf_pd_pit_version;\n mt...,DATASTEP,2021-07-27 10:34:25.730,0.007
2,3,proc sql noprint;\n create table __mtf_pd_pit...,SQL,2021-07-27 10:34:25.740,2.941
3,4,proc sql noprint;\n create table __mtf_pd_pit...,SQL,2021-07-27 10:34:28.680,2.490
4,5,proc sql noprint;\n select count(*) into :__i...,SQL,2021-07-27 10:34:31.170,0.128
...,...,...,...,...,...
158,159,proc transpose data=WORK.__MTF_PD_PIT_ODR_WID...,TRANSPOSE,2021-07-27 10:36:36.550,0.659
159,160,proc sql noprint _method;\n create table work...,SQL,2021-07-27 10:36:37.210,0.208
160,161,proc datasets lib = work nolist noprint memty...,DATASETS,2021-07-27 10:36:37.420,0.468
161,162,proc datasets lib = work nolist noprint memty...,DATASETS,2021-07-27 10:36:37.890,0.044


#### 1.3. Extract Edges

In [7]:
def df_to_connections(df):
    
    #-------------------------------------------
    # 3.1 Extract Table info
    # 3.1.a Get data
    df_tables = df.loc[(df['Task Type'].isin(['DATASET', 'OPENTIME', 'TASKSTARTTIME',  'JOBENDTIME']))].copy()   

    # 3.1.b # Extract table name and type
    df_tables['Table'] = df_tables[['Text','Task Type']].apply(lambda x: x[0].split()[3]  if x[1]=='DATASET' 
                                                                         else (x[0].split()[1]   if x[1]=='OPENTIME' 
                                                                               else '')
                                                               , axis = 1)
    df_tables['Table type'] = df_tables['Table'].str.split('.').str[-1]                          # Extract table type
    df_tables['Table'] = df_tables['Table'].apply(lambda x: '.'.join(x.split('.')[:-1]))         # Extract table name

    # 3.1.c Define table type (Input, Output, Update)
    df_tables['Dataset type'] = df_tables[['Text', 'Task Type']].apply(lambda x: x[0].split()[1]  if x[1]=='DATASET'
                                                                                 else ''
                                                                       , axis = 1)
    df_tables['Dataset type'] = np.where(df_tables['Task Type'].isin(['OPENTIME']),
                                          df_tables['Dataset type'].shift(),                     # Inherit table type
                                          df_tables['Dataset type'])
    
    # 3.1.d Add sub-step indicator
    df_tables = df_tables.reset_index(drop=True)
    df_tables['Lag Task ID'] = df_tables['Task ID'].shift()
    df_tables['Lag Dataset type'] = df_tables['Dataset type'].shift()
    sub_step_list = [1]
    for index, row in df_tables.iterrows():
        if (row['Task ID'] != row['Lag Task ID']):
            sub_step_list.append(1)                       # Reset sub-step
        elif ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'INPUT')) | \
             ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'UPDATE')) | \
             ((row['Lag Dataset type'] == 'UPDATE') & (row['Dataset type'] == 'INPUT')):
            sub_step_list.append(sub_step_list[index] +1) # Increase sub-step
        else:
            sub_step_list.append(sub_step_list[index])    # Keep sub-step
    df_tables['SubTask ID'] = sub_step_list[1:]

    # 3.1.e Correct case for updated tables
    df_tables['Dataset type'] = df_tables['Dataset type'].replace({'UPDATE': 'UPDATE INPUT'})
    df_update = df_tables.loc[df_tables['Dataset type'].isin(['UPDATE INPUT'])].copy()
    df_update['Dataset type'] = df_update['Dataset type'].replace({'UPDATE INPUT': 'UPDATE OUTPUT'})
    df_tables = df_tables.append(df_update).sort_values(['Task ID', 'SubTask ID', 'Dataset type'])
    
    # 3.1.f Add node index
    node_id_dict = {}
    node_id_list = []
    for index, row in df_tables.iterrows():
        # If update or output:
        if (row['Dataset type'] == 'OUTPUT') | (row['Dataset type'] == 'UPDATE OUTPUT') :
            # create node ID
            node_id = str(row['Task ID']) +':'+ \
                      str(row['SubTask ID']) +':'+ \
                      row['Table'] +':'+ \
                      row['Table type']   
            # update column
            node_id_list.append(node_id)       
            # update dict
            node_id_dict[row['Table']] = node_id                                                                    

        # If source table:
        elif row['Table'] not in node_id_dict:
            # create node ID
            node_id = '0:0:' + \
                      row['Table']  +':'+ \
                      row['Table type']
            # update column
            node_id_list.append(node_id)                                                           
            # update dict
            node_id_dict[row['Table']] = node_id  
        # If input:
        else: 
            # use previous node ID 
            node_id_list.append(node_id_dict[row['Table']])                                                         
    df_tables['Node Id'] = node_id_list

    # 3.1.g Remove leftover tables
    df_tables.drop(['Table type','Lag Task ID','Lag Dataset type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.2. Prepare info about Input tables
    df_input = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                             (df_tables['Dataset type'].isin(['INPUT', 'UPDATE INPUT']))].copy()
    df_input = df_input.rename(columns={'Table': 'Input Table',
                                        'Node Id': 'Source ID'})
    df_input.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.3. Prepare info about Output tables
    df_output = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                              (df_tables['Dataset type'].isin(['OUTPUT', 'UPDATE OUTPUT']))].copy()
    df_output = df_output.rename(columns={'Table': 'Output Table',
                                          'Node Id': 'Target ID'})
    df_output.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.4. Prepare info about Time Calculations
    # 3.4.a Get Data
    df_time = df_tables.loc[df_tables['Task Type'].isin(['OPENTIME', 'TASKSTARTTIME', 'JOBENDTIME'])].copy()

    # 3.4.b Add time
    df_time['Start Time'] = df_time[['Text','Task Type']].apply(lambda x: x[0].split()[2].replace('DATE:', '')  if x[1]=='OPENTIME'   # Inherit table type
                                                                          else x[0].split()[1].replace('DATE:', '')
                                                                , axis = 1)
    df_time['Start Time']  = pd.to_datetime(df_time['Start Time'], format='%d%b%Y:%H:%M:%S.%f')

    # 3.4.c Remove columns and rows
    df_time.drop(['Table','Text','Task Type','Dataset type','Node Id'], inplace=True, axis=1)
    df_time = df_time.drop_duplicates(subset=['Task ID', 'SubTask ID'], keep='first')

    # 3.4.d Calculate time for each step
    df_time['Elapsed Time'] = (df_time['Start Time'].shift(-1) - df_time['Start Time']).dt.total_seconds()
    df_time['Elapsed Time'] = df_time['Elapsed Time'].round(2)

    #-------------------------------------------
    # 3.5. Output
    # 3.5.a Merge the results for input and output tables + Elapsed time
    df = df_input.merge(df_output, on=['Task ID', 'SubTask ID'], how = 'outer')\
                 .merge(df_time, on=['Task ID', 'SubTask ID'], how = 'left')\
                 .sort_values(['Task ID', 'SubTask ID'])\
                 .reset_index(drop = True)

    # 3.5.b Fill missing for specific cases
    df['Input Table'] = df['Input Table'].fillna('No Input')
    df['Output Table'] = df['Output Table'].fillna('_null_')

    # 3.5.c Correct Start time
    df['Start Time'] = df['Start Time'].fillna(df['Start Time'].shift()) 
    df['Elapsed Time'] = df['Elapsed Time'].fillna(0)

    # 3.5.d Correct Target node ID
    df['Target ID'] = df['Target ID'].fillna(df['Task ID'].astype('str') +':'+ df['SubTask ID'].astype('str')+ ':No Output:Empty') 

    # 3.5.e Correct Source node ID
    df['Source ID'] = df[['Input Table','Task ID','SubTask ID','Source ID']].apply(lambda x: str(x[1])+':'+str(int(x[2])-1)+':No Input:Empty' if x[0]=='No Input' 
                                                                                  else x[3], axis = 1)
    
    # 3.5.f Reorder columns
    df = df[['Task ID','SubTask ID', 'Source ID','Target ID', 'Input Table','Output Table', 'Start Time','Elapsed Time']]
    
    return df

In [8]:
df_connections = df_to_connections(df_log)
df_connections

,Task ID,SubTask ID,Source ID,Target ID,Input Table,Output Table,Start Time,Elapsed Time
0,1,1,1:0:No Input:Empty,1:1:WORK.__MTF_PD_PIT_VERSION:DATA,No Input,WORK.__MTF_PD_PIT_VERSION,2021-07-27 10:34:25.720,0.01
1,2,1,1:1:WORK.__MTF_PD_PIT_VERSION:DATA,2:1:No Output:Empty,WORK.__MTF_PD_PIT_VERSION,_null_,2021-07-27 10:34:25.730,0.01
2,3,1,0:0:INPUT.PD_VIEW:DATA,3:1:WORK.__MTF_PD_PIT_DISTINCT_RATING1:DATA,INPUT.PD_VIEW,WORK.__MTF_PD_PIT_DISTINCT_RATING1,2021-07-27 10:34:25.740,2.93
3,3,1,0:0:INPUT.PD_VIEW:DATA,3:1:WORK.'SASTMP-000000348':UTILITY,INPUT.PD_VIEW,WORK.'SASTMP-000000348',2021-07-27 10:34:25.740,2.93
4,3,2,0:0:INPUT.PD_RR_RATING:DATA,3:2:WORK.__MTF_PD_PIT_DISTINCT_RATING2:DATA,INPUT.PD_RR_RATING,WORK.__MTF_PD_PIT_DISTINCT_RATING2,2021-07-27 10:34:28.670,0.00
...,...,...,...,...,...,...,...,...
286,160,1,159:1:WORK.__MTF_PD_PIT_ODR_LONG_DIS:DATA,160:1:WORK.TEST_PD_DIS_IN:DATA,WORK.__MTF_PD_PIT_ODR_LONG_DIS,WORK.TEST_PD_DIS_IN,2021-07-27 10:36:37.210,0.21
287,160,1,100:1:WORK.__MTF_PD_PIT_MARGINAL_BOTH_PD:DATA,160:1:WORK.TEST_PD_DIS_IN:DATA,WORK.__MTF_PD_PIT_MARGINAL_BOTH_PD,WORK.TEST_PD_DIS_IN,2021-07-27 10:36:37.210,0.21
288,161,1,161:0:No Input:Empty,161:1:WORK.'SASTMP-000000636':UTILITY,No Input,WORK.'SASTMP-000000636',2021-07-27 10:36:37.420,0.47
289,162,1,162:0:No Input:Empty,162:1:WORK.'SASTMP-000000638':UTILITY,No Input,WORK.'SASTMP-000000638',2021-07-27 10:36:37.890,0.04


## 2. Graph data

In [9]:
# Function for defining positions of tables
def get_coords(G):
    
    #--------------------------------------------------------------------
    # get source nodes 
    source_nodes = sorted([node for node in G.nodes() if G.in_degree(node) == 0])
    
    #--------------------------------------------------------------------
    # Get nodes in traversal order
    list_nodes = []
    for i in range(len(source_nodes)):
        # Define source node
        source = source_nodes[i]
        # Get nodes in depth order
        list_nodes_depth = list(nx.dfs_preorder_nodes(G, source=source, depth_limit=None))
        # Extend main list
        list_nodes.extend(list_nodes_depth)
        
    #--------------------------------------------------------------------
    # Add elements if missed
    list_nodes_full = sorted(list(G.nodes))
    list_nodes.extend(list_nodes_full)

    #--------------------------------------------------------------------
    # Create df
    df_nodes = pd.DataFrame(list_nodes, columns =['Node'])
    df_nodes['Task ID'] = df_nodes['Node'].str.split(':').str[0].astype('float')
    df_nodes['SubTask ID'] = df_nodes['Node'].str.split(':').str[1].astype('float')
    df_nodes = df_nodes.drop_duplicates(subset=['Node'], keep='first')
    
    #--------------------------------------------------------------------
    # Assign y
    df_nodes = df_nodes.reset_index(drop=True)
    df_nodes['Lag Node'] = df_nodes['Node'].shift()
    df_nodes['Lag Task ID'] = df_nodes['Task ID'].shift()
    df_nodes['Lag SubTask ID'] = df_nodes['SubTask ID'].shift()
    y_list = [0] 
    for index, row in df_nodes.iterrows():
        # Get predecessors
        predecessors = [c for c in G.predecessors( row['Node'] )]
        if (row['Lag Node'] in predecessors):
            if (row['Task ID'] == row['Lag Task ID'])  & (row['SubTask ID'] == row['Lag SubTask ID']):
                y_list.append(y_list[index] + 1)
            else: 
                y_list.append(y_list[index])
        else:
            y_list.append(y_list[index] + 1)
    df_nodes['y'] = y_list[1:]
    
    #--------------------------------------------------------------------
    # Reorder nodes
    df_nodes = df_nodes.sort_values(by=['Task ID','SubTask ID'])

    #--------------------------------------------------------------------
    # Assign x
    df_nodes = df_nodes.reset_index(drop=True)
    df_nodes['Lag Task ID'] = df_nodes['Task ID'].shift()
    df_nodes['Lag SubTask ID'] = df_nodes['SubTask ID'].shift()
    x_list = [-2] 
    for index, row in df_nodes.iterrows():
        # Move from task to task
        if (row['Task ID'] != row ['Lag Task ID']):
            x_list.append(x_list[index] + 2)
        # Move from subtask to subtask        
        elif (row['Task ID'] == row ['Lag Task ID']) & (row['SubTask ID'] != row ['Lag SubTask ID']):
            x_list.append(x_list[index] + 1)  
        else:
            x_list.append(x_list[index])
    df_nodes['x'] = x_list[1:]

    #--------------------------------------------------------------------
    # Coords dict
    coords = df_nodes[['Node', 'x' , 'y']].set_index('Node').T.to_dict('list')
    
    return coords

In [10]:
# Function for extracting data on tables
def get_node_df(G):  

    node_dict = {'node_x':[],
                 'node_y':[],
                 'node_label':[],
                 'step_id':[],
                 'node_group':[],
                 'node_color':[],
                 'node_shape':[]}
    
    for node in G.nodes():  
        #--------------------------------------------------------------------
        # Node Coords
        x, y = G.nodes[node]['coords']
        node_dict['node_x'].append(x)
        node_dict['node_y'].append(y)   

        #--------------------------------------------------------------------
        # Node name
        table_name = node.split(':')[2]
        node_dict['node_label'].append(table_name)  
        
        #--------------------------------------------------------------------
        # Step ID
        node_dict['step_id'].append(node.split(':')[0])  
        
        #--------------------------------------------------------------------
        # Table Type
        table_type = node.split(':')[3]       
        
        #--------------------------------------------------------------------
        # Predesessors
        predecessors = [c.split(':')[2] for c in G.predecessors(node)]

        #--------------------------------------------------------------------
        # Node shape and color
        # a) No Input node
        if table_name == 'No Input':
            node_dict['node_group'].append('No Input')  
            node_dict['node_shape'].append('diamond-cross')  
            node_dict['node_color'].append('grey')  
        # b) No Output node
        elif table_name == 'No Output':
            node_dict['node_group'].append('No Output')  
            node_dict['node_shape'].append('square-cross')  
            node_dict['node_color'].append('grey')
        # c) Temporary tables
        elif (table_type == 'UTILITY') | ('SORTTMP' in table_name):
            node_dict['node_group'].append('Technical Table')  
            node_dict['node_shape'].append('star-square-dot')  
            node_dict['node_color'].append('silver')
        # d) Input node
        elif G.in_degree(node) == 0:
            node_dict['node_group'].append('Input Table')  
            node_dict['node_shape'].append('diamond')  
            node_dict['node_color'].append('gold')
        # e) Output node
        elif G.out_degree(node) == 0:
            node_dict['node_group'].append('Output Table')  
            node_dict['node_shape'].append('square')  
            node_dict['node_color'].append('cyan')
        # f) Updated node            
        elif table_name in predecessors:
            node_dict['node_group'].append('Updated Table')  
            node_dict['node_shape'].append('cross')  
            node_dict['node_color'].append('orange')
        # g) Internal nodes
        else:
            node_dict['node_group'].append('Internal Table')  
            node_dict['node_shape'].append('circle')  
            node_dict['node_color'].append('blue')

    node_df = pd.DataFrame(node_dict)
    node_df['step_id'] = node_df['step_id'].astype('int')
    
    return node_df

In [11]:
# Function for extracting data on connections
def get_edge_df(G):
    
    edge_dict = {'edge_x':[],
                 'edge_y':[],
                 'node_x':[],
                 'node_y':[],
                 'text_x':[],
                 'text_y':[],
                 'step_id':[],
                 'text_color_values':[],
                 'text_labels':[]}

    for edge in G.edges():
        #--------------------------------------------------------------------
        # Get Coords
        x0, y0 = G.nodes[edge[0]]['coords']
        x1, y1 = G.nodes[edge[1]]['coords']
        x_mean = round( (x0+x1)/2, 4)
        y_mean = round( (y0+y1)/2, 4)   

        #--------------------------------------------------------------------
        # Shift central points        
        shift = (( (x1-x0)**2 + (y1-y0)**2 )**0.5) * 0.05    
        # If Horizontal line
        if x0 == x1:
            x_mean = x_mean + shift
        # If Vertical line            
        elif y0 == y1:
            y_mean = y_mean - shift

        #--------------------------------------------------------------------
        # Colorbar values
        color_value = G.edges[edge[0],edge[1]]['Elapsed Time'] 

        #--------------------------------------------------------------------
        # Hover text
        text_label = 'Task ID: '+str(G.edges[edge[0],edge[1]]['Task ID'])\
                        +' '+\
                        'SubTask ID: '+str(G.edges[edge[0],edge[1]]['SubTask ID'])\
                        +'<br> '+\
                        'Input: '+str(G.edges[edge[0],edge[1]]['Input Table'])\
                        +'<br> '+\
                        'Output: '+str(G.edges[edge[0],edge[1]]['Output Table'])\
                        +'<br> '+\
                        'Elapsed Time: '+str(G.edges[edge[0],edge[1]]['Elapsed Time'])

        #--------------------------------------------------------------------
        # output
        edge_dict['edge_x'].append(x0)
        edge_dict['edge_y'].append(y0)    
        edge_dict['node_x'].append(x1)
        edge_dict['node_y'].append(y1)  
        edge_dict['text_x'].append(x_mean)
        edge_dict['text_y'].append(y_mean)  
        edge_dict['step_id'].append(G.edges[edge[0],edge[1]]['Task ID']) 
        edge_dict['text_color_values'].append(color_value)   
        edge_dict['text_labels'].append(text_label) 

    edge_df = pd.DataFrame(edge_dict)

    return edge_df

In [12]:
# Function for generating a networkx Graph
def get_graph_data(df):

    # Create graph
    G = nx.from_pandas_edgelist(df,
                                'Source ID','Target ID',
                                ['Task ID','SubTask ID','Input Table','Output Table','Start Time','Elapsed Time'],
                                create_using=nx.DiGraph())
    print(nx.info(G))

    # Define positions of tables
    coords = get_coords(G)
    
    # Add positions to the graph
    nx.set_node_attributes(G, coords, 'coords') 

    # Create table for connections and label text
    edge_df = get_edge_df(G)

    # Create table for nodes
    node_df = get_node_df(G)

    # Combine the two tables
    df_graph = edge_df.merge(node_df, on=['step_id', 'node_x', 'node_y'], how = 'outer')
    
    # Fill empty values 
    # 1. Edge coords
    df_graph['edge_x'] = df_graph['edge_x'].replace(np.nan, 'None')
    df_graph['edge_y'] = df_graph['edge_y'].replace(np.nan, 'None')
    # 2. Text coords
    df_graph['text_x'] = df_graph['text_x'].replace(np.nan, 'None')
    df_graph['text_y'] = df_graph['text_y'].replace(np.nan, 'None')
    # 3. Labels
    df_graph['text_labels'] = df_graph['text_labels'].fillna('')
    # 4. Color
    df_graph['text_color_values'] = df_graph['text_color_values'].fillna(0)
    
    # Get color codes
    norm = matplotlib.colors.Normalize(vmin=min(df_graph['text_color_values']), 
                                       vmax=max(df_graph['text_color_values']), 
                                       clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap='RdYlGn_r')
    df_graph['edge_hex'] = df_graph['text_color_values'].apply(lambda x: matplotlib.colors.to_hex(mapper.to_rgba(x), keep_alpha=False))
    
    df_graph = df_graph
    return df_graph.sort_values(by=['step_id'])

df_graph = get_graph_data(df_connections)

DiGraph with 241 nodes and 291 edges


## 3. Dash plot

### 3.1. Network plot

#### 3.1.a Node Trace

In [13]:
# Function for drawing table positions
def get_node_trace(df):
    
    node_trace = go.Scatter(x = df['node_x'], 
                            y = df['node_y'],
                            mode = 'markers',
                            # Marker
                            marker=dict(symbol = df['node_shape'], 
                                        color = df['node_color'],
                                        size = 10),
                            marker_line_color='black', 
                            marker_line_width=0.5,
                            # Text
                            text = df['node_label'],
                            textposition = 'top center',                            
                            hovertemplate = '%{text}',
                            # Legend
                            showlegend=False,
                            # Meta
                            meta = df['step_id']
                           )
    
    return node_trace

In [14]:
# network plot
node_trace = get_node_trace(df_graph)

#### 3.1.b. Edge Trace

In [15]:
# Function for drawing table connections
def get_edge_trace(df):
    
    edge_trace = []
    edge_trace_outline = []    
    for index, row in df.iterrows():
        # Outline 
        edge_trace_outline.append(go.Scatter(x = [row['edge_x'], row['text_x'], row['node_x'], None],
                                             y = [row['edge_y'], row['text_y'], row['node_y'], None],
                                             mode = 'lines', 
                                             # line
                                             line_shape = 'spline',
                                             line = dict(width=3, 
                                                         dash='dot', 
                                                         color='grey'),
                                             # Text
                                             hoverinfo = 'none',
                                             # Legend
                                             showlegend=False,
                                             # Meta
                                             meta = row['step_id'] 
                                            )
                         )
        # Line
        edge_trace.append(go.Scatter(x = [row['edge_x'], row['text_x'], row['node_x'], None],
                                     y = [row['edge_y'], row['text_y'], row['node_y'], None],
                                     mode = 'lines', 
                                     # line
                                     line_shape = 'spline',
                                     line = dict(width=1.5, 
                                                 dash='dot', 
                                                 color=row['edge_hex']),
                                     # Text
                                     hoverinfo = 'none',
                                     # Legend
                                     showlegend=False,
                                     # Meta
                                     meta = row['step_id'] 
                                    )
                         )
        
    return edge_trace, edge_trace_outline

In [16]:
edge_trace, edge_trace_outline = get_edge_trace(df_graph)

#### 3.1.c Text trace

In [17]:
# Function for drawing connection info
def get_text_trace(df):
    
    text_trace = go.Scatter(x=df['text_x'], 
                            y=df['text_y'], 
                            # Marker
                            mode = 'markers', 
                            marker_symbol = 'hexagram',
                            marker=dict(showscale=True, 
                                        colorscale='RdYlGn', 
                                        reversescale=True,
                                        size = 8, 
                                        color=df['text_color_values'],
                                        colorbar=dict(thickness=15,
                                                      title='Execution time (s)',
                                                      xanchor='left',
                                                      titleside='right')
                                       ),                            
                            marker_line_color='black', 
                            marker_line_width=0.5,
                            # Text
                            text = df['text_labels'],  
                            textposition = 'top center',
                            hovertemplate = '%{text}',
                            # Legend
                            showlegend=False,
                            # Meta
                            meta = df['step_id']                           
                           )
    
    return text_trace

In [18]:
text_trace = get_text_trace(df_graph)

#### 3.1.d Network Graph

In [19]:
def draw_network(edge_trace_outline, edge_trace, node_trace, text_trace):
    fig = go.Figure(edge_trace_outline + edge_trace + [node_trace] + [text_trace]) 
    fig.update_layout(xaxis=dict(zeroline=False,
                                 showticklabels=False),
                      yaxis=dict(showgrid=False, 
                                 zeroline=False,
                                 showticklabels=False,
                                 autorange='reversed'),
                      legend=dict(orientation='h',
                                  x=1, y=1.02,
                                  xanchor='right', yanchor='bottom'),
                      margin=dict(t=0, b=10,
                                  l=10, r=0)
                     )
    
    return fig

In [20]:
fig_net = draw_network(edge_trace_outline, edge_trace, node_trace, text_trace)

#### 3.1.e Legend

In [21]:
def draw_legend():
    fig = go.Figure(go.Scatter(x=[0,0,0,0,0,0,0],
                               y=['No Input','No Output','Technical Table','Input Table','Output Table','Updated Table','Internal Table'] ,
                               mode = 'markers',
                               # Marker
                               marker=dict(symbol = ['diamond-cross','square-cross','star-square-dot','diamond','square','cross','circle'],
                                           color = ['grey','grey','silver','gold','cyan','orange','blue'],
                                           size = 10),
                               marker_line_color='black',
                               marker_line_width=0.5,
                               text = ['No Input','No Output','Technical Table','Input Table','Output Table','Updated Table','Internal Table'],
                               # Legend
                               showlegend=False)
                          )
    fig.update_layout(xaxis=dict(showgrid=False,
                                 showticklabels=False),
                      yaxis=dict(showgrid=False),
                      xaxis_side="top",
                      margin=dict(t=0, b=10,
                                  l=0, r=0),
                      width = 100
                     )
    return fig

In [22]:
fig_legend = draw_legend()

### 3.2 Table

#### 3.2.a Table filtering

In [23]:
# Operator dict
operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]

#======================================================================
# Filter query syntax split
def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part

                return name, operator_type[0].strip(), value

    return [None] * 3


#### 3.2.b Draw Table

In [24]:
def draw_table(df):
    table = dash_table.DataTable(id='table',
                                 columns=[{'name': i, 'id': i} for i in df.columns],
                                 data=df.to_dict('records'),
                                 # Filtering
                                 filter_action='native',
                                 filter_query='',
                                 # Styles
                                 style_header={'fontWeight': 'bold',
                                               'backgroundColor': 'rgb(48, 84, 150)',
                                               'color': 'white'},
                                 style_filter={'backgroundColor': 'rgb(142, 169, 219)',
                                               'color': 'white'},
                                 style_cell={'backgroundColor': 'rgb(217, 225, 242)',
                                             'textAlign': 'left', 'color': 'black',
                                             'width': '150px', 'minWidth': '180px', 'maxWidth': '180px',
                                             'whiteSpace': 'pre-line'},
                                 # Sorting
                                 sort_action='native',
                                 sort_mode='multi',
                                 # Column size
                                 style_cell_conditional=[{'if': {'column_id': 'Task ID'}, 'width': '5%'},
                                                         {'if': {'column_id': 'Code'}, 'width': '75%'},
                                                         {'if': {'column_id': 'Procedure'}, 'width': '5%'},
                                                         {'if': {'column_id': 'Start Time'}, 'width': '10%'},
                                                         {'if': {'column_id': 'Elapsed Time'}, 'width': '5%'}],
                                 # Table size
                                 fixed_rows={'headers': True},
                                 style_table={'overflowY': 'scroll', 
                                              'border': 'thin lightgrey solid'}, 
                                )
    return table

In [25]:
fig_table = draw_table(df_code)

### 3.3 Interactions

#### 3.3.a Scatter -> Table interaction

In [26]:
def update_query(selectedData):
    # Get list of selected Step IDs
    x_values = []
    for elements in selectedData['points']:
        if 'meta' in elements:
            if elements['meta'] not in x_values:
                x_values.append( int(elements['meta']) )
    # Generate filter query
    query =''
    if len(x_values) != 0:
        # 2.1 Formulate query
        for filter_value in x_values:
            query = query + '{Task ID} = ' + str(filter_value) + ' or '
        # 2.2. Remove last or
        query = ' '.join(query.split(' ')[:-2])
    return query

#### 3.3.b Table  -> Scatter interaction

In [27]:
def update_scatter(filter_query):
    # 1. Make copy of df and figure
    dff = copy.copy(df_code) 
    fig_copy = copy.copy(fig_net) 

    # 2. Fetch step ids based on filter query
    filtering_expressions = filter_query.split(' && ')
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)
        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]   
        elif operator == 'datestartswith':
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]
    step_list = list(dff['Task ID'])
    
    # 3. Select figure traces that are in the filtered data
    selected_points = []
    fig_data = fig_copy['data']
    for i in range(len(fig_data)):
        meta = fig_data[i]['meta']
        if isinstance(meta, int):
            if meta in step_list:
                selected_points.append(i)
    
    # 4. Update figure
    fig_copy.update_traces(selectedpoints = selected_points)

    return fig_copy

### 3.4. Dash Plot

In [ ]:
# Server start
app = dash.Dash()

#======================================================================
# Dash plot
app.layout = html.Div([
    #--------------------------------------------------------------
    # 1. First row - Reset Button
    html.Div(children=[html.Button('Reset Graphs', id='reset_button', n_clicks=0)],
             className='row',
             style={'height':'5%', 'display':'inline-block', 'vertical-align':'top', 'horizontal-align':'center'}), 
    
    #--------------------------------------------------------------
    # 2. Second row - Legend and Network
    html.Div(children=[
        # 1.1 first column of first row - Legend
        html.Div(children=[dcc.Graph(id='legend', figure=fig_legend)],
                 style={'width':'10%', 'display':'inline-block', 'vertical-align':'top', 'horizontal-align':'center'}),
        # 1.2 second column of first row - Network
        html.Div(children=[dcc.Graph(id='network', figure=fig_net)],
                 style={'width':'90%', 'display':'inline-block', 'vertical-align':'top', 'horizontal-align':'center'}),
    ], className='row'),
    
    #--------------------------------------------------------------
    # 3. Third row - Table
    html.Div(children=[fig_table], 
             className='row', 
             style={'width':'100%', 'vertical-align':'top', 'horizontal-align':'center'})
])

# ======================================================================
# Add Scatter-Table-Reset interactions
@app.callback([Output('reset_button','n_clicks'), Output('table', 'filter_query'), Output('network', 'selectedData'), Output('network', 'figure')],
              [Input('reset_button','n_clicks'), Input('table', 'filter_query'), Input('network', 'selectedData')])
def update_figures(n_clicks, filter_query, selectedData):
    
    # 1. If reset is clicked - restart table and scatter
    if n_clicks is not None and n_clicks > 0:  
        return 0, '', None, fig_net  
    
    # 2. If filter is applied to Network -> update Table
    elif selectedData is not None:
        selected_query = update_query(selectedData)
        if filter_query != '':            
            full_query = '(' + filter_query + ') and (' + selected_query + ')' # Data selected based on query and scatter
            return dash.no_update, full_query, dash.no_update, dash.no_update
        else:
            return dash.no_update, selected_query, None, dash.no_update
        
    # 3. If filter is applied to Table - update Network
    elif filter_query is not None and filter_query != '': 
        fig_updated = update_scatter(filter_query)
        return dash.no_update, dash.no_update,  None, fig_updated
    
    # 4. If filter query is reset - update Network
    elif filter_query == '' and selectedData is None:
        return dash.no_update, dash.no_update,  None, fig_net        
    
    # 5. No change
    else:
        raise dash.exceptions.PreventUpdate
        
#======================================================================
app.run_server(debug=False)    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1622715601.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1622715601.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1622715601.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_table/bundle.v4_11_3m1622715601.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1622715601.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1622715604.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2022 13:40:02] "GET /_dash-compone